# EDAした結果よりデータを変形するアレ
開発時以外は**`Run ALL`以外しない**

## テスト通らないカラムを扱っちゃだめよ

In [1]:
# `python -m unittest discover tests -v`と等価
import unittest
TEST_RESULT = unittest.TextTestRunner(verbosity=2).run(
    unittest.TestLoader().discover("tests")
)

test_error (test_int_sex.testIntSex)
エラーになるケースのテスト ... ok
test_main (test_int_sex.testIntSex)
メインのテスト ... ok
test_error (test_is_pair.testIsPair)
エラーを出す場合のテスト ... ok
test_example (test_is_pair.testIsPair)
例なデータでのテスト ... ok
test_Age_not_found (test_under_10_years_old.TestUnder10YearsOld)
Ageカラムが存在しない場合のエラーのテスト ... ok
test_all_age (test_under_10_years_old.TestUnder10YearsOld)
0-200才までテスト ... ok
test_example (test_under_10_years_old.TestUnder10YearsOld)
例なデータのテスト（正例） ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.033s

OK


## データの生成

In [2]:
import pandas as pd

from transforms.column_under_10_years_old import column_under_10_years_old
from transforms.column_is_pair import column_is_pair
from transforms.column_int_sex import column_int_sex

def generate(raw: pd.DataFrame) -> pd.DataFrame:
    preprocessed =  pd.DataFrame({
        "PassengerId": raw.PassengerId,
        "IntSex": column_int_sex(raw),
        "Pclass": raw.Pclass,
        "Under10YearsOld": column_under_10_years_old(raw),
        "IsPair": column_is_pair(raw),
        "Parch": raw.Parch
    })

    # ターゲットの列があれば含める（TRAIN向け）
    if "Survived" in raw:
        return preprocessed.assign(Survived=raw.Survived)
    
    return preprocessed


SyntaxError: invalid syntax (2041444115.py, line 10)

In [ ]:
if (
    len(TEST_RESULT.failures) == 0
    and len(TEST_RESULT.errors) == 0
    and TEST_RESULT.testsRun != 0
):
    TRAIN, TEST = (
        generate(pd.read_csv("data/train.csv")),
        generate(pd.read_csv("data/test.csv")),
    )
    train_filename = "data/preprocessed_train.csv"
    test_filename = "data/preprocessed_test.csv"

    print("following dataframe will be saved (or OVERWRITED)")
    print("-----------", train_filename, "-----------")
    display(TRAIN)

    print("-----------", test_filename, "-----------")
    display(TEST)

    if input("continue to save (overwrite)? (y/N)"):
        TRAIN.to_csv(train_filename, index=None, header=True)
        TEST.to_csv(test_filename, index=None, header=True)
else:
    print("test is not completed successfully or nothing tested.")


following dataframe will be saved (or OVERWRITED)
----------- data/preprocessed_train.csv -----------


,Sex,Pclass,Under10YearsOld,IsPair,Parch,Survived
0,male,3,0,1,0,0
1,female,1,0,1,0,1
2,female,3,0,0,0,1
3,female,1,0,1,0,1
4,male,3,0,0,0,0
...,...,...,...,...,...,...
886,male,2,0,0,0,0
887,female,1,0,0,0,1
888,female,3,0,1,2,0
889,male,1,0,0,0,1


----------- data/preprocessed_test.csv -----------


,Sex,Pclass,Under10YearsOld,IsPair,Parch
0,male,3,0,0,0
1,female,3,0,1,0
2,male,2,0,0,0
3,male,3,0,0,0
4,female,3,0,1,1
...,...,...,...,...,...
413,male,3,0,0,0
414,female,1,0,0,0
415,male,3,0,0,0
416,male,3,0,0,0
